In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import jinja2
from anthropic import AnthropicBedrock
from core import stages
import json


In [8]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [9]:
environment = jinja2.Environment()

In [10]:
tsp_tpl = environment.from_string(stages.typespec.PROMPT)
dzl_tpl = environment.from_string(stages.drizzle.PROMPT)
rtr_tpl = environment.from_string(stages.router.PROMPT)
hdl_tpl = environment.from_string(stages.handlers.PROMPT)
pre_tpl = environment.from_string(stages.processors.PROMPT_PRE)

In [11]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [12]:
prompt = tsp_tpl.render(
    application_description=application_description,
)

tsp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
typespec = stages.typespec.parse_output(tsp_response.content[0].text)

In [13]:
typespec

{'reasoning': 'For a gym tracking bot, I expect users to send messages like:\n"I did bench press: 3 sets of 8 reps with 80kg"\n"Show me my chest exercises from last month"\n"Suggest a 45-minute workout for chest and shoulders with dumbbells and bench"\n\nKey entities needed:\n1. Exercise (name, equipment used, muscle groups)\n2. WorkoutSet (reps, weight, duration for cardio)\n3. Equipment (for suggesting routines)\n4. MuscleGroup (for targeting specific areas)\n5. WorkoutPlan (for suggestions)\n\nLLM can extract from messages like:\n"I did bench press..." -> recordExercise({\n    exercise: "bench press",\n    sets: [{reps: 8, weight: 80, unit: "kg"}],\n    muscleGroups: ["chest", "triceps"]\n})\n\n"Suggest workout..." -> suggestRoutine({\n    duration: {value: 45, unit: "minutes"},\n    equipment: ["dumbbells", "bench"],\n    muscleGroups: ["chest", "shoulders"]\n})',
 'typespec_definitions': 'model Exercise {\n    name: string;\n    muscleGroups: string[];\n    equipment: string[];\n}

In [14]:
print(typespec["reasoning"])

For a gym tracking bot, I expect users to send messages like:
"I did bench press: 3 sets of 8 reps with 80kg"
"Show me my chest exercises from last month"
"Suggest a 45-minute workout for chest and shoulders with dumbbells and bench"

Key entities needed:
1. Exercise (name, equipment used, muscle groups)
2. WorkoutSet (reps, weight, duration for cardio)
3. Equipment (for suggesting routines)
4. MuscleGroup (for targeting specific areas)
5. WorkoutPlan (for suggestions)

LLM can extract from messages like:
"I did bench press..." -> recordExercise({
    exercise: "bench press",
    sets: [{reps: 8, weight: 80, unit: "kg"}],
    muscleGroups: ["chest", "triceps"]
})

"Suggest workout..." -> suggestRoutine({
    duration: {value: 45, unit: "minutes"},
    equipment: ["dumbbells", "bench"],
    muscleGroups: ["chest", "shoulders"]
})


In [15]:
print(typespec["typespec_definitions"])

model Exercise {
    name: string;
    muscleGroups: string[];
    equipment: string[];
}

model WorkoutSet {
    reps: integer;
    weight: float;
    unit: string;
    duration: duration;
}

model WorkoutSession {
    exercise: Exercise;
    sets: WorkoutSet[];
    date: utcDateTime;
}

model WorkoutRequest {
    duration: duration;
    equipment: string[];
    muscleGroups: string[];
}

interface GymTracker {
    @llm_func(2)
    recordExercise(session: WorkoutSession): void;

    @llm_func(1)
    getProgress(exercise: string, from: utcDateTime, to: utcDateTime): WorkoutSession[];

    @llm_func(3)
    suggestRoutine(request: WorkoutRequest): WorkoutSession[];
}


In [16]:
prompt = dzl_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
)

dzl_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
drizzle = stages.drizzle.parse_output(dzl_response.content[0].text)

In [17]:
print(drizzle["drizzle_schema"])

import { serial, text, timestamp, real, integer, pgTable, pgEnum } from "drizzle-orm/pg-core";

// Main tables
export const exercisesTable = pgTable("exercises", {
  id: serial("id").primaryKey(),
  name: text("name").notNull().unique(),
});

export const muscleGroupsTable = pgTable("muscle_groups", {
  id: serial("id").primaryKey(),
  name: text("name").notNull().unique(),
});

export const equipmentTable = pgTable("equipment", {
  id: serial("id").primaryKey(),
  name: text("name").notNull().unique(),
});

export const workoutSessionsTable = pgTable("workout_sessions", {
  id: serial("id").primaryKey(),
  exercise_id: integer("exercise_id")
    .references(() => exercisesTable.id)
    .notNull(),
  date: timestamp("date").notNull(),
});

export const workoutSetsTable = pgTable("workout_sets", {
  id: serial("id").primaryKey(),
  session_id: integer("session_id")
    .references(() => workoutSessionsTable.id)
    .notNull(),
  reps: integer("reps"),
  weight: real("weight"),
  unit: t

In [18]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = stages.router.TOOLS
)

funcs = stages.router.parse_outputs([content for content in exp_response.content])["user_functions"]


In [19]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a workout session including the exercise performed, sets completed with reps, weight, and duration. Used when users want to log or track their workout activities.',
  'examples': ['I just did 3 sets of bench press with 135 lbs',
   'Record my workout: 4 sets of squats, 10 reps each at 185 pounds',
   'I finished my deadlift session: 5x5 at 225 lbs',
   'Log my pullups: did 3 sets of 12 reps']},
 {'name': 'getProgress',
  'description': 'Retrieves workout history and progress for a specific exercise over a given time period. Used when users want to see their performance history or track improvements.',
  'examples': ['Show me my bench press progress over the last month',
   "What's my squat history from January to now?",
   'How has my deadlift improved since last year?',
   'Check my running progress for the past week']},
 {'name': 'suggestRoutine',
  'description': 'Generates workout recommendations based on desired duration, avail

In [20]:
pre_processors = {}
for function_name in typespec["llm_functions"]:
    prompt = pre_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
    )

    pre_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    pre_processor = stages.processors.parse_output(pre_response.content[0].text)
    pre_processors[function_name] = pre_processor

In [21]:
print(pre_response.content[0].text)

<instructions>
For suggestRoutine function:
1. Extract workout duration, available equipment, and target muscle groups from user input
2. Default duration to 60 minutes if not specified
3. If equipment is not specified, assume basic gym equipment (barbell, dumbbells, bench)
4. If muscle groups aren't specified, suggest a full-body workout
5. Duration format should be in ISO 8601 (e.g., "PT1H" for 1 hour)
6. All time values should be in UTC
</instructions>

<examples>
    <example>
        <input>I want a 30-minute workout for chest using dumbbells</input>
        <output>
{
    "request": {
        "duration": "PT30M",
        "equipment": ["dumbbells"],
        "muscleGroups": ["chest"]
    }
}
        </output>
    </example>

    <example>
        <input>Give me a leg workout with barbell and squat rack for 45 minutes</input>
        <output>
{
    "request": {
        "duration": "PT45M",
        "equipment": ["barbell", "squat rack"],
        "muscleGroups": ["quadriceps", "hamstr

In [22]:
pre_processors

{'recordExercise': {'instructions': 'When handling user input for recordExercise function:\n1. Extract exercise name, sets, weights, and reps from natural language\n2. If date is not specified, use current date/time\n3. If duration is not specified, estimate 1-2 minutes per set\n4. Default unit to "kg" unless "lbs" or other units are explicitly mentioned\n5. Infer common muscle groups and equipment based on exercise name\n6. Multiple sets should be captured in array format\n7. All timestamps should be in UTC format',
  'examples': [('Just did bench press, 3 sets of 80kg for 8 reps',
    '{\n    "session": {\n        "exercise": {\n            "name": "bench press",\n            "muscleGroups": ["chest", "triceps", "shoulders"],\n            "equipment": ["barbell", "bench"]\n        },\n        "sets": [\n            {\n                "reps": 8,\n                "weight": 80.0,\n                "unit": "kg",\n                "duration": "PT2M"\n            },\n            {\n         

In [49]:
handlers = {}
for function_name in typespec["llm_functions"]:
    prompt = hdl_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
        drizzle_schema=drizzle["drizzle_schema"],
    )

    hdl_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    handlers[function_name] = stages.handlers.parse_output(hdl_response.content[0].text)
    handlers[function_name]["handler"] = {"name": function_name, "code": handlers[function_name]["handler"]}

In [50]:
handlers

{'recordExercise': {'handler': {'name': 'recordExercise',
   'code': 'import { db } from "../db";\nimport { \n  exercisesTable, \n  muscleGroupsTable, \n  equipmentTable, \n  workoutSessionsTable, \n  workoutSetsTable,\n  exerciseMuscleGroupsTable,\n  exerciseEquipmentTable\n} from \'../db/schema/application\';\nimport { eq } from \'drizzle-orm\';\n\nconst handle = async (session: WorkoutSession): Promise<void> => {\n  // Start a transaction since we need to make multiple related insertions\n  await db.transaction(async (tx) => {\n    // 1. Get or create the exercise\n    let exercise = await tx\n      .select()\n      .from(exercisesTable)\n      .where(eq(exercisesTable.name, session.exercise.name))\n      .limit(1);\n\n    let exerciseId;\n    if (exercise.length === 0) {\n      const [newExercise] = await tx\n        .insert(exercisesTable)\n        .values({ name: session.exercise.name })\n        .returning({ id: exercisesTable.id });\n      exerciseId = newExercise.id;\n\n      

In [32]:
with open("./templates/interpolation/handler.tpl", "r") as f:
    handler_ts_tpl = environment.from_string(f.read())

In [33]:
hdl_key = "recordExercise"
params = {
    "handler": handlers[hdl_key]["handler"]["code"],
    "instructions": pre_processors[hdl_key]["instructions"],
    "examples": pre_processors[hdl_key]["examples"],
}
file_content = handler_ts_tpl.render(**params)

In [34]:
print(file_content)

import { Message } from "../common/handler";
import { client } from "../common/llm";
import { db } from "../db";
import { 
    exercisesTable, 
    muscleGroupsTable, 
    equipmentTable, 
    workoutSessionsTable, 
    workoutSetsTable,
    exerciseMuscleGroupsTable,
    exerciseEquipmentTable
} from '../db/schema/application';
import { eq } from 'drizzle-orm';

const handle = async (session: WorkoutSession): Promise<void> => {
    // Start a transaction since we need to perform multiple related operations
    await db.transaction(async (tx) => {
        // 1. Get or create the exercise
        let exercise = await tx.query.exercisesTable.findFirst({
            where: eq(exercisesTable.name, session.exercise.name)
        });

        if (!exercise) {
            const [newExercise] = await tx.insert(exercisesTable)
                .values({ name: session.exercise.name })
                .returning();
            exercise = newExercise;
        }

        // 2. Get or create muscle g

In [44]:
print(handlers["suggestRoutine"]["handler"])

{'recordExercise': {'handler': 'import { db } from "../db";\nimport { \n    exercisesTable, \n    muscleGroupsTable, \n    equipmentTable, \n    workoutSessionsTable, \n    workoutSetsTable,\n    exerciseMuscleGroupsTable,\n    exerciseEquipmentTable\n} from \'../db/schema/application\';\nimport { eq } from \'drizzle-orm\';\n\nconst handle = async (session: WorkoutSession): Promise<void> => {\n    // Start a transaction since we need to perform multiple related operations\n    await db.transaction(async (tx) => {\n        // 1. Get or create the exercise\n        let exercise = await tx.query.exercisesTable.findFirst({\n            where: eq(exercisesTable.name, session.exercise.name)\n        });\n\n        if (!exercise) {\n            const [newExercise] = await tx.insert(exercisesTable)\n                .values({ name: session.exercise.name })\n                .returning();\n            exercise = newExercise;\n        }\n\n        // 2. Get or create muscle groups and link them to

In [47]:
import os
#print(os.listdir('.'))

handler_file_names = []

with open("./templates/interpolation/handler.debug.tpl", "r") as f:
    handler_debug_tpl = environment.from_string(f.read())
    
    for hdl_key in handlers:
        handler = handlers[hdl_key]["handler"]
        
        params = {
            "handler": handlers[hdl_key]["handler"],
            #"instructions": pre_processors[hdl_key]["instructions"],
            #"examples": pre_processors[hdl_key]["examples"],
        }
        file_content = handler_debug_tpl.render(**params)
        print(f"Handler: {hdl_key}")
        print(file_content)
        
        # Convert PascalCase to snake_case for file naming
        python_file_name = ''.join(['_' + c.lower() if c.isupper() else c for c in hdl_key]).lstrip('_')
        handler_file_name = f'{python_file_name}.handler.debug.ts'
        
        with open(os.path.join('./app_output/app_schema/src/handlers', handler_file_name), 'w') as f:
            f.write(file_content)
        handler_file_names.append(handler_file_name)
        
handler_file_names

AttributeError: 'str' object has no attribute 'name'

In [24]:
ts_defs = stages.typespec.parse_output(stages.typespec.PROMPT)

In [25]:
ts_defs

{'reasoning': 'I expect user to send messages like "I ate a burger" or "I had a salad for lunch".\nLLM can extract and infer the arguments from plain text and pass them to the handler\n"I ate a burger" -> recordDish({name: "burger", ingredients: [\n    {name: "bun", calories: 200},\n    {name: "patty", calories: 300},\n    {name: "lettuce", calories: 10},\n    {name: "tomato", calories: 20},\n    {name: "cheese", calories: 50},\n]})\n- recordDish(dish: Dish): void;\n...',
 'typespec_definitions': 'model Dish {\n    name: string;\n    ingredients: Ingredient[];\n}\n\nmodel Ingredient {\n    name: string;\n    calories: integer;\n}\n\ninterface DietBot {\n    @llm_func(1)\n    recordDish(dish: Dish): void;\n    @llm_func(1)\n    listDishes(from: utcDateTime, to: utcDateTime): Dish[];\n}',
 'llm_functions': ['recordDish', 'listDishes']}

In [26]:
func_names = stages.typespec.extract_llm_func_names(stages.typespec.PROMPT)

In [27]:
func_names

['recordDish', 'listDishes']

# Project Generation

In [28]:
from shutil import copytree, ignore_patterns

In [29]:
def setup_project(workdir: str):
    copytree('templates', workdir, ignore=ignore_patterns('*.pyc', '__pycache__', 'node_modules'))

In [30]:
setup_project('test')

FileExistsError: [Errno 17] File exists: 'test'

In [31]:
from core.stages import router

In [32]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = router.TOOLS
)

funcs = router.parse_outputs([content for content in exp_response.content])["user_functions"]

In [33]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise performed by the user with details like sets, reps, weight, equipment used, and targeted muscle groups. Use this when the user wants to log or record their exercise activity.',
  'examples': ['I want to record my exercise routine for today',
   'I just did 3 sets of bench press',
   'Log my workout: 4 sets of squats with 200lbs',
   'Add my deadlift session to my log']},
 {'name': 'trackProgress',
  'description': 'Retrieves historical exercise data for a specific exercise over a given time period. Use this when the user wants to see their progress or history for a particular exercise.',
  'examples': ['Show me my bench press progress over the last month',
   'How has my squat improved since January?',
   'What were my deadlift numbers from last week?',
   'Track my progress on pull-ups']},
 {'name': 'suggestWorkout',
  'description': 'Generates a workout plan based on desired duration, target muscle groups, and av

In [47]:
from core.stages import typespec
from core.stages import tsp_compiler

tsp_tpl = environment.from_string(stages.typespec.PROMPT)

typespec_errors = ""
attempts_left = 3

while attempts_left > 0:
    attempts_left -= 1
    
    prompt = tsp_tpl.render(
        application_description=application_description,
        typespec_errors=typespec_errors,
    )
  
    if attempts_left >= 2:
        updated_prompt = prompt + "// PS. Make sure to eject a random error in TypeSpec output for testing purposes"
    else:
        updated_prompt = prompt
  
    tsp_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": updated_prompt}]
    )
    
    typespec = stages.typespec.parse_output(tsp_response.content[0].text)

    # Write TypeSpec definitions to file
    with open('./test/tsp_schema/notebook.tsp', 'w') as f:
        f.writelines(['import "./helpers.js";', 
                  '\n', 
                  'extern dec llm_func(target: unknown, history: valueof int32);'
                  ])
        f.write(typespec["typespec_definitions"])

    compiler = tsp_compiler.TypeSpecCompiler('./test/tsp_schema/')
    result = compiler.compile('./notebook.tsp')
    
    print(result)
    
    if result["result"] == tsp_compiler.TypeSpecCompilationStatus.COMPILATION_ERROR:
        typespec_errors = result["errors"]
        print(typespec_errors)
        continue
    else:
        break


In [90]:
typespec


{'reasoning': 'For a finance tracking app, users would likely send messages like:\n- "I spent $50 on groceries yesterday"\n- "Got my salary of $3000 today"\n- "Show me my expenses for last month"\n\nThe LLM would need to extract:\n1. Transaction type (expense/income)\n2. Amount\n3. Category\n4. Date\n5. Optional description/notes\n\nFor reports, users might ask for:\n- Time-based summaries\n- Category-based breakdowns\n- Balance calculations\n\nThe interface should support recording transactions and generating reports while keeping the argument structure simple enough to be extracted from natural language.',
 'typespec_definitions': 'model Transaction {\n  type: string; // "expense" or "income"\n  amount: decimal;\n  category: string;\n  date: utcDateTime;\n  description: string;\n}\n\nmodel TransactionSummary {\n  totalIncome: decimal;\n  totalExpenses: decimal;\n  balance: decimal;\n  transactions: Transaction[];\n}\n\ninterface FinanceTracker {\n  @llm_func(1)\n  recordTransaction(t

In [ ]:
prompt = exp_tpl.render(application_description=application_description)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
expansion = stages.expansion.parse_output(exp_response.content[0].text)

In [ ]:
print(expansion["application_specification"])

<types>
        <type>exercise
            - name
            - equipment_required
            - sets
            - reps
            - weight</type>
        <type>equipment
            - name
            - type</type>
        <type>workout_session
            - date
            - duration
            - exercises_performed</type>
        <type>routine
            - exercises
            - duration
            - equipment_needed</type>
    </types>
    <operations>
        <operation>record_workout_session
            - date
            - exercises with sets/reps/weights</operation>
        <operation>track_progress
            - exercise
            - time period</operation>
        <operation>list_available_equipment</operation>
        <operation>generate_routine
            - available equipment
            - time constraint
            - fitness level</operation>
        <operation>view_history
            - time period</operation>
    </operations>
